In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="5ADzvGdu4blAVrKyRPYy")
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(1)
dataset = version.download("yolov8")
                

In [ ]:
!pip install albumentations opencv-python numpy


In [ ]:
import cv2
import os
import albumentations as A
import numpy as np

# Define input and output paths
input_images = r"D:\project_test\development_and_analysis\football-players-detection-1\train\images"
input_labels = r"D:\project_test\development_and_analysis\football-players-detection-1\train\labels"

output_images = r"D:\project_test\development_and_analysis\football-players-detection-1\train\augmented_images"
output_labels = r"D:\project_test\development_and_analysis\football-players-detection-1\train\augmented_labels"

# Create output folders if they don’t exist
os.makedirs(output_images, exist_ok=True)
os.makedirs(output_labels, exist_ok=True)

# Define augmentation pipeline (applies to both images & bounding boxes)
augmentations = A.Compose([
    A.RandomRotate90(p=0.5),  # Rotate 90° randomly
    A.HorizontalFlip(p=0.5),  # Flip horizontally
    A.VerticalFlip(p=0.2),  # Flip vertically
    A.RandomBrightnessContrast(p=0.3),  # Adjust brightness/contrast to highlight the ball
    A.MotionBlur(blur_limit=3, p=0.2),  # Simulate motion blur
    A.GaussianBlur(blur_limit=3, p=0.2),  # Slight blur to account for low-quality frames
    A.RandomScale(scale_limit=0.3, p=0.5),  # Increase size of small objects
    A.RandomResizedCrop((640, 640), scale=(0.6, 1.0), ratio=(0.75, 1.33), p=0.3),  # ✅ Fixed RandomResizedCrop
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.2),  # Improves contrast for small objects
    A.ToGray(p=0.1)  # Convert some images to grayscale (simulates different lighting conditions)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# Loop through each image and its corresponding label
for img_file in os.listdir(input_images):
    if img_file.endswith((".jpg", ".png", ".jpeg")):
        # Load image
        img_path = os.path.join(input_images, img_file)
        image = cv2.imread(img_path)

        # Load YOLO label
        label_path = os.path.join(input_labels, img_file.replace(".jpg", ".txt").replace(".png", ".txt"))
        if not os.path.exists(label_path):
            continue  # Skip images without labels

        with open(label_path, "r") as f:
            lines = f.readlines()

        # Parse bounding boxes
        bboxes = []
        class_labels = []
        for line in lines:
            parts = line.strip().split()
            class_id = int(parts[0])
            bbox = list(map(float, parts[1:]))  # X_center, Y_center, Width, Height
            bboxes.append(bbox)
            class_labels.append(class_id)

        # Apply augmentation
        augmented = augmentations(image=image, bboxes=bboxes, class_labels=class_labels)

        # Save augmented image
        aug_img_path = os.path.join(output_images, f"aug_{img_file}")
        cv2.imwrite(aug_img_path, augmented["image"])

        # Save augmented labels
        aug_label_path = os.path.join(output_labels, f"aug_{img_file.replace('.jpg', '.txt').replace('.png', '.txt')}")
        with open(aug_label_path, "w") as f:
            for bbox, cls in zip(augmented["bboxes"], augmented["class_labels"]):
                f.write(f"{cls} " + " ".join(map(str, bbox)) + "\n")

print("✅ Data augmentation with bounding boxes completed!")


In [ ]:
!pip install ultralytics

